In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import threading
import websocket

from collections import deque

import time
import datetime
import json
import logging
import logging.config

logging.config.fileConfig('./ini/logconfig.ini')
logger = logging.getLogger("KULOKO")

TRADE_ENDPOINT='wss://api.coin.z.com/ws/public/v1'
# websocketを使ってtickerをリアルタイム取得
websocket.enableTrace(True) #trace ON

class Trade(object):
    def __init__(self, sym,maxlen=100):
        self.sym = sym
        self.maxlen =maxlen
        self.queue = deque([],self.maxlen)
        self.connect()

    def connect(self):
        self.ws = websocket.WebSocketApp(
            TRADE_ENDPOINT,
            on_message = self.on_message,on_open=self.on_open,
            on_error = self.on_error, on_close = self.on_close)
        logger.info("Socket Connected")
    
    def subscribe(self):
        self.ws.keep_running = True 
        self.thread = threading.Thread(target=lambda: self.ws.run_forever())
        self.thread.daemon = True
        self.thread.start()
        logger.info("Start to subscribe")

    def is_connected(self):
        return self.ws.sock and self.ws.sock.connected

    def disconnect(self):
        self.ws.keep_running = False
        self.ws.close()
        logger.info("Socket closed")

    def get(self):
        queue_len = len(self.queue)
        return [ self.queue.popleft() for _ in range(queue_len)]

    def clean_data(self, return_data):
        for _i in range(return_data):
            return_data[_i]['timestamp'] =return_data[_i]['timestamp']
            return_data[_i]['price'] =return_data[_i]['price']
            return_data[_i]['size'] =return_data[_i]['size']
        return return_data


    def on_message(self, message):
        logger.info('Received:{0}'.format(message))
        self.queue.append(json.loads(message))
        if len(self.queue) > self.maxlen:
            logger.warn("Message queue is full. Old item are discarded")

    def on_error(self, error):
        logger.error('Error. Try reconnect {0}'.format(error))
        self.disconnect()
        time.sleep(0.5)
        self.connect()

    def on_close(self):
        logger.info('Websocket disconnected')

    def on_open(self):
        message = {
            "command": "subscribe",
            "channel": "trades",
            "symbol": self.sym 
        }
        self.ws.send(json.dumps(message))
        logger.info('Socket opened')

trade = Trade('BTC')
    # while True:
    #     print(bfrt.get())
    #     time.sleep(0.5)

In [41]:
trade.connect()

In [43]:
trade.subscribe()


In [44]:
trade.get()

[{'channel': 'trades',
  'price': '1201694',
  'side': 'SELL',
  'size': '0.001',
  'symbol': 'BTC',
  'timestamp': '2020-10-13T15:03:45.612Z'}]

In [39]:
trade.disconnect()

In [1]:
import json
import websocket

websocket.enableTrace(True)
ws = websocket.WebSocketApp('wss://api.coin.z.com/ws/public/v1')

def on_open(self):
    message = {
        "command": "subscribe",
        "channel": "trades",
        "symbol": "BTC"
    }
    ws.send(json.dumps(message))

def on_message(self, message):
    print(message)

ws.on_open = on_open
ws.on_message = on_message

ws.run_forever()

NameError: name 'a' is not defined